In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer
from transformers import TextIteratorStreamer, StoppingCriteria, StoppingCriteriaList

import torch

from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

import os
import requests
import re
from collections import Counter
from threading import Thread
from dotenv import load_dotenv



/home/ubuntu/anaconda3/envs/llm_eds/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## User Query

In [27]:

query = "What precautions should pregnant women with EDS consider adopting?"


## Load Zebra-Llama from Huggingface

In [28]:
model = AutoModelForCausalLM.from_pretrained(
    "zebraLLAMA/zebra-Llama-v0.2",
    revision="main",
    cache_dir="/home/ubuntu/.hf_cache",
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True,
    rope_scaling={"type": "dynamic", "factor": 8.0}
)

tokenizer = AutoTokenizer.from_pretrained(
    "zebraLLAMA/zebra-Llama-v0.2",
    revision="main",
    cache_dir="/home/ubuntu/.hf_cache",
    padding_side="left",
    add_eos_token=True,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

device = next(model.parameters()).device


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.06s/it]


## LLM text generation hyperparameters (adjust as needed)

In [29]:
MAX_TOKENS = 2048
TOP_P = 0.95
LLM_TOP_K = 50


## Set RAG parameters

In [30]:

RAG_BASE_URI = "https://zebra-llama-rag.onrender.com"
RAG_ENDPOINT = "/search"
RAG_URI = RAG_BASE_URI + RAG_ENDPOINT
RAG_TOP_K = 1



## Set Embedding Model

In [31]:

#Replace the env file and key names based on the configuration you provided

load_dotenv(os.path.join(os.path.expanduser('~'), '.openai_config.env'))
embed_model = OpenAIEmbedding(
    model=os.environ.get('RAG_EMBEDDING_MODEL'),
    api_key=os.environ.get('HACKATHON_API_KEY'),
)
Settings.embed_model = embed_model




## Set System Prompt

In [32]:
system_prompt = '''
You are an expert AI assistant specializing in Ehlers-Danlos syndrome (EDS). Your role is to provide comprehensive, accurate, and well-structured answers about EDS. You will be provided with a prompt that has two components such as "User message" and "Context". Follow these guidelines to address the prompt:

- In the first paragraph, begin with a broad overview that directly addresses the "User message".
- In the second paragraph, provide detailed information mainly by using the given "Context". Also use your trained knowledge about EDS to supplement the assertions. If you don't see relevant information in the context, always mention that in your response and stick on to your own internal knowledge to answer the question.
- Answer in multiple paragraphs and be comprehensive in your answer
- Structure your response logically:
     a) Start with a general answer to the question.
     b) Provide specific examples or details, always with proper citations. 
     c) You can find the citations at the end of each "Context" para marked as '(Ref: '. Do not use any references that do not contain a DOI, and do not use references that contain just numbers in square brackets. Here are examples of references to avoid: [ 1 ], [5, 6, 8], etc.
- If mentioning specific studies or cases, clearly state their relevance to the main question and provide proper context. 
- When answering questions based on provided context, do not use phrases like 'The context provided' or 'In the provided context' in your responses.
- In the last paragraph, conclude with a brief summary of the key points.
IMPORTANT: If you receive a question unrelated to Ehlers-Danlos Syndrome (EDS), respond directly by stating that the question is not related, without providing any additional context or explanations. For example, if the question is "Who is the actor in the movie titanic" and even if it has any EDS context given in the "Context", your answer should be like "Sorry, this question is not related to EDS and I cannot address that."

Some examples for your reference are given below:
Example 1:
Context : Context not related to EDS
User message: What is the most popular song of Taylor Swift?
Your response : Sorry, this question is not directly related to EDS. So, I cannot answer that.

Example 2:
Context : The analysis of the hEDS patients’ ﬁbroblasts showed that there is a greater expression
of AQP9 gene and down regulation of IL6 gene. AQP9 and IL6 are involved in the
inﬂammatory process, since EDS patients frequently show chronic disease, AQP9 and IL6
could be use as possible diagnostic biomarkers.
In addition, PRL may have a role in inﬂammation and their serum levels could suggest
a clinical situation of chronic inﬂammation that is frequent in EDS patients. Scientiﬁc data
show the up-regulation of SELENBP1 in schizophrenic patients. Considering that EDS
patients suffer from psychiatric disorders, it would be interesting to dose SELENBP1 in
serum EDS patients.
User message: What are the biomarkers associated with EDS?
Your response: Biomarkers are measurable indicators that can be used to assess the presence or severity of a disease. For Ehlers-Danlos Syndrome (EDS), research has identified several potential biomarkers that could help in diagnosing or understanding the disease better. EDS is a group of connective tissue disorders characterized by hypermobility, skin elasticity, and tissue fragility. Identifying reliable biomarkers is crucial for effective diagnosis and management, given the complex and varied symptoms associated with different types of EDS.
One of the potential biomarkers in EDS is the **Aquaporin 9 (AQP9) gene**, which has been shown to be overexpressed in the fibroblasts of hypermobile EDS (hEDS) patients. AQP9 plays a role in the inflammatory process, and its increased expression may be linked to the chronic inflammation often observed in EDS patients. Additionally, the **Interleukin-6 (IL6) gene**, which is also involved in inflammation, has been found to be downregulated in these patients. This dysregulation of genes related to the inflammatory response suggests that both AQP9 and IL6 could serve as useful diagnostic markers in identifying inflammatory states associated with EDS.
Furthermore, **Prolactin (PRL)** has been suggested as another potential biomarker. Elevated serum levels of PRL may indicate a state of chronic inflammation, which is frequently observed in EDS patients. Another interesting biomarker is **SELENBP1** (selenium-binding protein 1), which is known to be upregulated in patients with schizophrenia. Considering the higher prevalence of psychiatric disorders in EDS patients, the measurement of SELENBP1 levels could be relevant in the context of EDS-related psychiatric manifestations. These findings, while still in the early stages of research, could pave the way for better diagnostic tools and therapeutic targets for EDS (Ref: 10.3390/ijms221810149).
In summary, biomarkers such as AQP9, IL6, PRL, and SELENBP1 are being explored in the context of EDS to help clarify the underlying mechanisms of the disease and potentially improve diagnostic accuracy. Further research and validation are necessary to establish these markers as definitive diagnostic tools.
'''


## Custom Functions and Classes

In [33]:
def get_rag_context(query, rag_uri, top_k=2):
    query_embedding = embed_model.get_text_embedding(query)
    response = requests.post(
        rag_uri,
        json={
            "query_embedding": query_embedding,
            "top_k": top_k
        }
    )
    if response.status_code == 200:
        return response.json()["context"]
    else:
        return ''

class RepetitionStoppingCriteria(StoppingCriteria):
    def __init__(self, tokenizer, repetition_threshold=3, window_size=200, min_length=20):
        self.tokenizer = tokenizer
        self.repetition_threshold = repetition_threshold
        self.window_size = window_size
        self.min_length = min_length
        self.generated_text = ""
        self.last_check_length = 0

    def __call__(self, input_ids, scores, **kwargs):
        new_text = self.tokenizer.decode(input_ids[0, self.last_check_length:], skip_special_tokens=True)
        self.generated_text += new_text
        self.last_check_length = len(input_ids[0])

        if len(self.generated_text) > self.window_size and self.check_repetition():
            return True

        return False

    def check_repetition(self):
        text = self.generated_text[-self.window_size:]
        sentences = re.split(r'[.!?]+', text)
        
        # Check for exact sentence repetitions
        sentence_counter = Counter(sentences)
        if any(count >= self.repetition_threshold for count in sentence_counter.values()):
            return True

        # Check for phrase repetitions
        phrases = self.get_phrases(text)
        phrase_counter = Counter(phrases)
        if any(count >= self.repetition_threshold and len(phrase) >= self.min_length for phrase, count in phrase_counter.items()):
            return True

        return False

    def get_phrases(self, text):
        words = text.split()
        phrases = []
        for i in range(len(words)):
            for j in range(i+1, len(words)+1):
                phrase = ' '.join(words[i:j])
                if len(phrase) >= self.min_length:
                    phrases.append(phrase)
        return phrases

    
def generate_text_using_stream(query, model):
    
    rag_context = get_rag_context(query, RAG_URI, top_k=RAG_TOP_K)
    prompt = f'''
    Context: {rag_context}
    User message: {query}
    '''
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_input = tokenizer(text, 
                            padding=True, 
                            return_tensors="pt").to(device)
    generation_kwargs = dict(
        **model_input,
        max_new_tokens=MAX_TOKENS,
        top_p=TOP_P,
        top_k=LLM_TOP_K,
        do_sample=True,
        streamer=streamer,
        stopping_criteria=StoppingCriteriaList([stopping_criteria])
    )
    
    # Start the generation in a separate thread
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    
    # Iterate over the generated text
    generated_text = ""
    for new_text in streamer:
        generated_text += new_text
        print(new_text, end="", flush=True)  # Print each piece of new text as it's generated
    
    thread.join()


## Text generation settings (adjust as needed)

In [34]:
repetition_stopping = RepetitionStoppingCriteria(tokenizer, repetition_threshold=3, window_size=200, min_length=20)
stopping_criteria = StoppingCriteriaList([repetition_stopping])
streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)


## Generate Response from Zebra-Llama

In [35]:

resp = generate_text_using_stream(query, model)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


For pregnant women with Ehlers-Danlos Syndrome (EDS), there are several precautions that should be considered to minimize risks and ensure a safe pregnancy. EDS, particularly the vascular type (vEDS), poses significant health risks for both the mother and the fetus due to the potential for arterial complications and vascular rupture. Therefore, careful planning and consideration are essential.

First and foremost, for women with vEDS, the primary concern is the risk of arterial complications, including arterial rupture and vascular events, which can be life-threatening. One of the most effective strategies for managing these risks is pre-implantation genetic diagnosis (PGD) through in vitro fertilization (IVF). This procedure involves genetic testing of embryos affected by the familial pathogenic variant (in this case, the COL3A1 gene) before transfer to the uterus, thus avoiding the risks associated with an affected fetus (Ref: 10.3389/fgene.2023.1147607). For women without vEDS, the 